# Prérequis
0. Installation des libairies necessaire pour le développement

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## Récupération d'un ensemble de données d'IRM cérébrale
Source : https://www.kaggle.com/datasets/navoneel/brain-mri-images-for-brain-tumor-detection

# Importation des bibliothèques

In [15]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset 
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import cv2